In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

In [5]:
def open_file():
    with open('ddata_min.pickle', 'rb') as f:
        f_data = pickle.load(f)
    return f_data

In [6]:
def coin_select():
    f_data = open_file()
    coin_name = str(input('Enter the coin name: '))
    s_coin_data = BTC_data = f_data[f_data.permno == coin_name].copy()
    s_coin_data = s_coin_data.reset_index()
    
    return s_coin_data

In [20]:
def RSI(period=30):
    s_coin_data = coin_select()
    period = period
    U = np.where(s_coin_data.prc.diff(1) > 0, s_coin_data.prc.diff(1), 0)
    D = np.where(s_coin_data.prc.diff(1) < 0, s_coin_data.prc.diff(1)*(-1), 0)
    AU = pd.DataFrame(U, index=s_coin_data.index).rolling(window=period).mean()
    AD = pd.DataFrame(D, index=s_coin_data.index).rolling(window=period).mean()
    RSI = AU / (AD+AU) * 100
    s_coin_data['RSI'] = RSI
    
    return s_coin_data

In [14]:
def MACD(n_short=30, n_long=60, n_signal=6):
    s_coin_data = coin_select()
    s_coin_data['EMA_short'] = s_coin_data['prc'].ewm(span=n_short, min_periods=n_short-1).mean()
    s_coin_data['EMA_long'] = s_coin_data['prc'].ewm(span=n_long, min_periods=n_long-1).mean()
    
    s_coin_data['MACD'] = s_coin_data['EMA_short'] - s_coin_data['EMA_long']
    s_coin_data['MACD_signal'] = s_coin_data['MACD'].ewm(span=n_signal, min_periods=n_signal-1).mean()
    s_coin_data['MACD_diff'] = s_coin_data['MACD'] - s_coin_data['MACD_signal']
    
    return s_coin_data

In [65]:
## RSI <= 30 and 0 < MACD_diff  ==>  BUY
## RSI >= 70 or 0 > MACD_diff  ==> SELL

def trading_func(initial_asset=100, sell_fee=0.0004, buy_fee=0.0004):
    s_coin_data = pd.merge(MACD(n_short=300, n_long=900, n_signal=6),
                           RSI(period=30))
    
    buy_price = 0
    total_profit = 0
    total_loss = 0
    win_count = 0
    lose_count = 0
    sell_signal = False
    asset = initial_asset
    sell_fee = sell_fee
    buy_fee = buy_fee
    position = False
    
    for i, row in s_coin_data.iterrows():
        date, price, rsi, macd_diff = row['date'], row['prc'], row['RSI'], row['MACD_diff']
    
        if sell_signal:
            # 수익률
            profit_pct = (price - buy_price) / buy_price - sell_fee
            s_coin_data.at[i, 'Profit'] = profit_pct * asset
            asset *= (1+profit_pct)

            #초기화
            buy_price = 0
            position = False
            sell_signal = False

            # 이익
            if profit_pct > 0:
                win_count += 1
                total_profit += profit_pct * asset

            # 손실
            else:
                lose_count += 1
                total_loss += profit_pct * asset


            print('SELL: {} | Sell price: {:7.2f} | Total asset: {:7.4f}'.format(date, price, asset), '\n')


        # Buy signal
        elif not(buy_price and position) and (rsi <= 30 and macd_diff >= 0):
            asset = asset * (1-buy_fee)
            buy_price = s_coin_data.prc[i]
            position = True

            print('BUY: {} | Buy price: {:7.2f} | Total asset: {:7.4f}'.format(date, buy_price, asset))

        # Sell signal
        elif (position and buy_price) and (rsi >= 70 or macd_diff < 0):
            sell_signal = True
            
        elif asset <= 0:
            break
        
    
    print('Final Asset: {:10.4f} | Winning Trade: {:7} | Losing Trade: {:7}'.
                format(asset, win_count, lose_count))
    
    return s_coin_data

In [66]:
if __name__ == "__main__":
    trading_func()

Enter the coin name:  BTC
Enter the coin name:  BTC


BUY: 2019-09-09 10:24:00 | Buy price: 10214.10 | Total asset: 99.9600
SELL: 2019-09-09 10:57:00 | Sell price: 10255.47 | Total asset: 100.3249 

BUY: 2019-09-09 21:55:00 | Buy price: 10289.34 | Total asset: 100.2848
SELL: 2019-09-09 21:57:00 | Sell price: 10286.20 | Total asset: 100.2140 

BUY: 2019-09-11 01:03:00 | Buy price: 10127.83 | Total asset: 100.1739
SELL: 2019-09-11 01:55:00 | Sell price: 10082.55 | Total asset: 99.6860 

BUY: 2019-09-11 11:45:00 | Buy price: 10080.54 | Total asset: 99.6461
SELL: 2019-09-11 12:46:00 | Sell price: 10079.21 | Total asset: 99.5931 

BUY: 2019-09-11 23:22:00 | Buy price: 10113.34 | Total asset: 99.5533
SELL: 2019-09-11 23:24:00 | Sell price: 10116.37 | Total asset: 99.5433 

BUY: 2019-09-12 14:30:00 | Buy price: 10276.19 | Total asset: 99.5035
SELL: 2019-09-12 15:33:00 | Sell price: 10334.01 | Total asset: 100.0236 

BUY: 2019-09-14 00:53:00 | Buy price: 10307.25 | Total asset: 99.9835
SELL: 2019-09-14 02:11:00 | Sell price: 10296.06 | Total asse

In [64]:
s_coin_data[-30:]

,date,permno,prc,tvol,mcap,EMA_short,EMA_long,MACD,MACD_signal,MACD_diff,RSI
949352,2021-06-29 00:30:00,BTC,34803.01,2111.001,6.494617e+11,34674.810354,34618.131324,56.679030,49.354487,7.324543,60.359138
949353,2021-06-29 00:31:00,BTC,34751.72,667.646,6.494617e+11,34686.642607,34628.026781,58.615826,51.206755,7.409071,62.100701
949354,2021-06-29 00:32:00,BTC,34775.84,567.003,6.494617e+11,34700.365283,34638.975909,61.389374,53.243279,8.146095,64.952481
949355,2021-06-29 00:33:00,BTC,34773.84,345.505,6.494617e+11,34711.669086,34648.965841,62.703244,55.135272,7.567972,63.013219
949356,2021-06-29 00:34:00,BTC,34755.02,172.646,6.494617e+11,34718.338457,34656.821705,61.516752,56.411568,5.105184,57.674960
949357,2021-06-29 00:35:00,BTC,34825.54,820.525,6.494617e+11,34734.831002,34669.319356,65.511646,58.231583,7.280062,57.060495
949358,2021-06-29 00:36:00,BTC,34907.74,1170.626,6.494617e+11,34761.432386,34686.980145,74.452242,61.475715,12.976526,60.696411
949359,2021-06-29 00:37:00,BTC,34909.89,881.513,6.494617e+11,34784.272019,34703.491986,80.780033,65.336579,15.443455,62.488056
949360,2021-06-29 00:38:00,BTC,34970.90,1331.414,6.494617e+11,34812.984016,34723.299987,89.684029,70.206069,19.477960,65.372723
949361,2021-06-29 00:39:00,BTC,35073.40,2345.013,6.494617e+11,34853.048014,34749.233321,103.814692,76.927794,26.886899,67.660518


In [59]:
s_coin_data[-30:]

,date,permno,prc,tvol,mcap,EMA_short,EMA_long,MACD,MACD_signal,MACD_diff,RSI
30000,2019-09-29 13:58:00,BTC,7986.25,4.775,1.481225e+11,7985.719287,7990.846144,-5.126857,-3.922063,-1.204794,47.998060
30001,2019-09-29 13:59:00,BTC,7995.62,7.039,1.481225e+11,7987.242473,7991.199763,-3.957289,-3.929108,-0.028181,52.276201
30002,2019-09-29 14:00:00,BTC,7990.00,14.497,1.481225e+11,7987.666708,7991.110891,-3.444183,-3.832123,0.387940,51.301115
30003,2019-09-29 14:01:00,BTC,7989.16,14.577,1.481225e+11,7987.896446,7990.966381,-3.069935,-3.679686,0.609750,48.060256
30004,2019-09-29 14:02:00,BTC,7982.95,20.325,1.481225e+11,7987.135454,7990.372575,-3.237121,-3.591173,0.354052,44.138355
30005,2019-09-29 14:03:00,BTC,7986.13,9.196,1.481225e+11,7986.980769,7990.058310,-3.077541,-3.488446,0.410905,42.753065
30006,2019-09-29 14:04:00,BTC,7983.40,8.238,1.481225e+11,7986.429881,7989.565102,-3.135221,-3.417801,0.282581,43.027355
30007,2019-09-29 14:05:00,BTC,7982.01,11.729,1.481225e+11,7985.749899,7989.005465,-3.255565,-3.385354,0.129789,46.255874
30008,2019-09-29 14:06:00,BTC,7986.08,11.915,1.481225e+11,7985.800684,7988.788764,-2.988079,-3.305899,0.317820,45.111851
30009,2019-09-29 14:07:00,BTC,7982.99,3.257,1.481225e+11,7985.368271,7988.359226,-2.990954,-3.242910,0.251956,44.098063


In [67]:
fastMAlen = input(12, minval=1, title="MACD fast moving average")

TypeError: raw_input() got an unexpected keyword argument 'minval'